<a href="https://colab.research.google.com/github/SupriyaRajaiahgari/GenAi/blob/main/nutriguide.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Step 1: Connect to Google Drive**

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

%cd gdrive/My Drive

Mounted at /content/gdrive
/content/gdrive/My Drive


**Step 2: Install all the necessary Packages**

In [ ]:
!pip install tensorflow
!pip install huggingface_hub
!pip install langchain
!pip install langchain-huggingface
!pip install pillow
!pip install streamlit
!pip install streamlit_jupyter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 100.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 74.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 59.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [ ]:
!pip install pyngrok

In [ ]:
# --- Import libraries ---
import tensorflow as tf
import numpy as np
import streamlit as st
import streamlit_jupyter as stj
from huggingface_hub import InferenceClient
import requests
from PIL import Image
from io import BytesIO
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import chain
from langchain_huggingface import HuggingFaceEndpoint
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel
import time
import os
import base64

In [ ]:
# --- Set your API token ---
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "Your_API_Key"

In [ ]:
# --- Initialize LLM ---
llm = HuggingFaceEndpoint(
    repo_id="microsoft/Phi-3-mini-4k-instruct",
    task="text-generation",    # <-- IMPORTANT
    max_new_tokens=512,
    temperature=0.7,
    repetition_penalty=1.03,
    huggingfacehub_api_token="Your_Key"
)
chat_nutrition_prompt = ChatPromptTemplate.from_template(
    '''
    Provide the nutrition information (Calories, Protein, Carbohydrates, Dietary Fiber, Sugars, Fat, Sodium, Potassium, Vitamin C, Vitamin B6) for {prediction} per 100 grams, Output the information as a concise, formatted list without repetition.
    '''
)

chat_health_benefits_prompt = ChatPromptTemplate.from_template(
    '''
    Provide the health benefits and considerations for {prediction}. Additionally, include practical tips for making {prediction} healthier. Keep the response focused on these two aspects only.
    '''
)

chat_recipes_prompt = ChatPromptTemplate.from_template(
    '''
    Tell me about the two most famous recipes for {prediction}. Include the ingredients only.
    '''
)


In [ ]:
# --- Helper functions ---
def load_and_prep_image(uploaded_file, img_shape=224):
    img = Image.open(uploaded_file)  # Open uploaded image
    img = img.resize((img_shape, img_shape))  # Resize image
    img = tf.convert_to_tensor(img, dtype=tf.float32)  # Convert to tensor
    return img

@chain
def predict_label(uploaded_file):
    img = load_and_prep_image(uploaded_file, img_shape=224)  # Preprocess image
    img = tf.expand_dims(img, axis=0)  # Add batch dimension
    pred = model.predict(img)  # Model prediction
    pred_class_index = np.argmax(pred, axis=1)[0]  # Get highest probability index
    pred_class_name = class_labels[pred_class_index]  # Convert index to class name
    return pred_class_name


In [ ]:
from google.colab import files

uploaded = files.upload()


Saving NewVersionModelOptimized40V2.zip to NewVersionModelOptimized40V2 (2).zip


In [ ]:
import zipfile
import os

with zipfile.ZipFile("NewVersionModelOptimized40V2 (2).zip", 'r') as zip_ref:
    zip_ref.extractall("/content/NewVersionModelOptimized40V2 (2).zip")


In [ ]:
import os
os.listdir('/content')


['.config', 'gdrive', 'NewVersionModelOptimized40V2 (1).zip', 'sample_data']

In [ ]:
import os
print(os.listdir('/content/NewVersionModelOptimized40V2 (2)'))

In [ ]:
# Load the .keras model
model = tf.keras.models.load_model('/content/NewVersionModelOptimized40V2/NewVersionModelOptimized40V2.keras')

# Check if the model is loaded successfully by printing its summary
model.summary()

In [ ]:



# --- Define your class labels ---
class_labels = {
    0: 'Baked Potato', 1: 'Burger', 2: 'Cake', 3: 'Chips', 4: 'Crispy Chicken', 5: 'Croissant',
    6: 'Dount', 7: 'Dragon Fruit', 8: 'Frise', 9: 'Hot Dog', 10: 'Jalapeno', 11: 'Kiwi',
    12: 'Lemon', 13: 'Lettuce', 14: 'Mango', 15: 'Onion', 16: 'Orange', 17: 'Pizza',
    18: 'Taquito', 19: 'apple', 20: 'banana', 21: 'beetroot', 22: 'bell pepper', 23: 'bread',
    24: 'cabbage', 25: 'carrot', 26: 'cauliflower', 27: 'cheese', 28: 'chilli pepper', 29: 'corn',
    30: 'crab', 31: 'cucumber', 32: 'eggplant', 33: 'eggs', 34: 'garlic', 36: 'grapes', 37: 'milk',
    38: 'salamon', 39: 'yogurt'
}

In [ ]:
# --- Setup LangChain chains ---
str_output_parser = StrOutputParser()

chain_label = predict_label
chain1 = chat_nutrition_prompt | llm | str_output_parser
chain2 = chat_health_benefits_prompt | llm | str_output_parser
chain3 = chat_recipes_prompt | llm | str_output_parser

chain_parallel = RunnableParallel({
    'chat_nutrition_prompt': chain1,
    'chat_health_benefits_prompt': chain2,
    'chat_recipes_prompt': chain3
})


In [ ]:
# --- Main App ---
with open('app.py', 'w') as f:
    f.write("""
import streamlit as st

def main():
    st.set_page_config(page_title="NutriGuide", layout="wide")
    st.title("NutriGuide 🍎")
    st.markdown("#### Analyze the nutrition, health benefits, and recipes for your food!")

    uploaded_file = st.file_uploader("Upload an image of food 🍽️", type=["jpg", "png", "jpeg"], label_visibility="collapsed")

    if uploaded_file is not None:
        with st.spinner('Analyzing your food...'):
            try:
                start_time = time.time()

                prediction = predict_label.invoke(uploaded_file)
                result = chain_parallel.invoke(prediction)

                end_time = time.time()
                elapsed_time = end_time - start_time

                st.markdown(f"⏱ **Processing Time:** {elapsed_time:.2f} seconds", unsafe_allow_html=True)
                st.write(f"### **Predicted Food Item:** {prediction}")
                st.image(uploaded_file, caption="Uploaded Image", width=350, use_container_width=True)

                st.subheader("🍏 **Nutrition Information:**")
                st.markdown(f"```{result['chat_nutrition_prompt']}```", unsafe_allow_html=True)

                st.subheader("💪 **Health Benefits and Tips:**")
                st.markdown(f"```{result['chat_health_benefits_prompt']}```", unsafe_allow_html=True)

                st.subheader("🍴 **Famous Recipes:**")
                st.markdown(f"```{result['chat_recipes_prompt']}```", unsafe_allow_html=True)

            except Exception as e:
                st.error(f"An error occurred: {e}")
                if __name__ == "__main__":
                    main()

                """)


In [ ]:
!pip install streamlit
!pip install pyngrok

In [ ]:
from pyngrok import ngrok

# Replace with your ngrok authtoken
ngrok.set_auth_token("Yuor_Key")

# Expose the Streamlit app on port 8501
#public_url = ngrok.connect(8501)

# Start the Streamlit app in the background
import os
os.system('streamlit run app.py &')

# Print the ngrok URL to access the Streamlit app
print('Streamlit app is available at:', public_url)